In [7]:
import findspark
findspark.init()
import pandas
import plotly.express as px
from datetime import datetime
import pytz
import org.apache.spark.sql.functions._

ModuleNotFoundError: No module named 'org'

In [19]:
import sys
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext
import pyspark.sql.functions as F

In [20]:
conf = SparkConf().setAppName("Dummy")
sc = SparkContext(conf=conf)

ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=Dummy, master=local[*]) created by __init__ at /tmp/ipykernel_27900/1097506723.py:2 

In [21]:
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

In [22]:
buses = sqlContext.read.format('csv').options(header='true', inferschema='true').load('hdfs://localhost:8020/user/borowskij/projekt/ztm/ztm.csv')

buses = buses.withColumn('Time',F.to_timestamp(F.col('Time'), 'yyyy-MM-dd HH:mm:ss').alias('Time'))
buses_pom = buses.groupBy('VehicleNumber').agg(F.max('Time').alias('Time'),F.first('Lat').alias('Lat'),
                                               F.first('Lon').alias('Lon'), F.first('Lines').alias('Lines'))
#buses = buses_pom.join(buses.drop('Time'),['VehicleNumber'],'leftsemi')

In [22]:

d=datetime.now()
from_d=d-timedelta(hours=23)
to_d = d+timedelta(hours=1)



buses.filter(F.col('Time')>from_d).filter(F.col('Time')<to_d).show()

+-----+----------+-------------+-------------------+----------+-------+
|Lines|       Lon|VehicleNumber|               Time|       Lat|Brigade|
+-----+----------+-------------+-------------------+----------+-------+
|  119|21.1670361|         1000|2022-01-15 13:53:21|52.2045151|      2|
|  213| 21.219732|         1001|2022-01-15 13:53:21|52.1571818|      4|
|  119|21.2055438|         1002|2022-01-15 13:53:22|52.2100788|      1|
|  213|21.1020918|         1013|2022-01-15 13:53:18|52.2140873|      1|
|  192|21.0685661|         1021|2022-01-15 13:53:22|52.1290812|      1|
|  195|21.0320024|         1022|2022-01-15 13:53:23|52.1490574|      2|
|  218| 21.023773|         1024|2022-01-15 13:53:23| 52.179863|      1|
|  218| 20.992741|         1025|2022-01-15 13:53:22| 52.188499|      2|
|  218| 21.013954|         1027|2022-01-15 13:53:23| 52.179901|      4|
|  119|21.0003044|         1030|2022-01-15 13:53:22|52.2061002|      4|
|  195| 21.024302|         1031|2022-01-15 13:53:23| 52.223282| 

In [17]:
from datetime import datetime,timedelta

In [18]:
d=datetime.now()

datetime.datetime(2022, 1, 15, 13, 51, 20, 958740)

In [87]:

def regen_map(): 
    buses = sqlContext.read.format('csv').options(header='true', inferschema='true').load('hdfs://localhost:8020/user/borowskij/projekt/ztm/ztm.csv')

    buses = buses.withColumn('Time',F.to_timestamp(F.col('Time'), 'yyyy-MM-dd HH:mm:ss').alias('Time'))
    buses_pom = buses.groupBy('VehicleNumber').agg(F.max('Time').alias('Time'),F.first('Lat').alias('Lat'),
                                               F.first('Lon').alias('Lon'), F.first('Lines').alias('Lines'))
    buses = buses_pom.toPandas()
    fig = px.scatter_mapbox(buses, lat="Lat", lon="Lon",color='Lines', hover_name="VehicleNumber",color_continuous_scale=px.colors.sequential.matter, size_max=20,
                        zoom=10, mapbox_style="open-street-map",title=f'Pozycje autobusów Warszawa {datetime.now(pytz.timezone("Europe/Warsaw")).strftime("%Y-%m-%d %H-%M")}' )
    fig.update_layout(margin={"r":2,"t":22,"l":2,"b":2},
                     title_x=0.5,
    title_y=0.99)
    fig.update_traces(marker={'size': 10})
    fig.show()
    

In [88]:
regen_map()